# forex-centuries: Interactive Exploration

This notebook walks through every major dataset in the forex-centuries project.
Run `python build.py` first to generate the derived data.

In [ ]:
import sys
from pathlib import Path

# Allow imports from project root
ROOT = Path.cwd().parent
sys.path.insert(0, str(ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import probplot, kurtosis

SOURCES = ROOT / "data" / "sources"
DERIVED = ROOT / "data" / "derived"
ANALYSIS = DERIVED / "analysis"
NORM = DERIVED / "normalized"

if not NORM.exists() or not ANALYSIS.exists():
    raise FileNotFoundError("Derived data not found. Run 'python build.py' first.")

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 5)
plt.rcParams["figure.dpi"] = 100

## 1. Yearly Panel (243 countries, 1500-2025)

The unified panel merges MeasuringWorth, Clio Infra, and the Global Macro Database
with priority MW > CI > GMD for overlapping (year, country) pairs.

In [ ]:
panel = pd.read_csv(NORM / "yearly_unified_panel.csv")
print(f"Shape: {panel.shape}")
print(f"Countries: {panel['country'].nunique()}")
print(f"Year range: {panel['year'].min()}-{panel['year'].max()}")
print(f"\nSource breakdown:")
print(panel["source"].value_counts())

# Longest series
series_len = panel.groupby("country").size().sort_values(ascending=False)
print(f"\nLongest series:")
for country, n in series_len.head(5).items():
    sub = panel[panel["country"] == country]
    print(f"  {country}: {n} years ({sub['year'].min()}-{sub['year'].max()})")

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))
for country, color in [("United Kingdom", "#DD8452"), ("Japan", "#55A868"), ("Switzerland", "#C44E52")]:
    sub = panel[panel["country"] == country].sort_values("year")
    ax.plot(sub["year"], sub["rate_per_usd"], label=country, color=color)
ax.set_xlabel("Year")
ax.set_ylabel("Rate per USD")
ax.set_title("Selected currencies vs USD (yearly)")
ax.legend()
ax.set_yscale("log")
plt.tight_layout()
plt.show()

## 2. Daily Data (23 currencies, 1971-2025)

FRED H.10 release: 23 daily currency pairs normalized to foreign-per-USD convention.

In [ ]:
daily = pd.read_csv(NORM / "fred_daily_normalized.csv")
print(f"Shape: {daily.shape}")
print(f"Currencies: {sorted(daily['currency'].unique())}")
print(f"Date range: {daily['date'].min()} to {daily['date'].max()}")

# Basic stats
stats = pd.read_csv(ANALYSIS / "daily_volatility_stats.csv")
print(f"\nVolatility stats (top 5 by kurtosis):")
print(stats[["currency", "annualized_volatility", "excess_kurtosis", "tail_ratio"]].head())

In [ ]:
# Pairwise correlation sample
corr = pd.read_csv(ANALYSIS / "daily_correlation_matrix.csv", index_col=0)
sample = ["EUR", "GBP", "JPY", "CHF", "BRL", "CNY"]
sample = [c for c in sample if c in corr.columns]
print("Daily log-return correlations (sample):")
print(corr.loc[sample, sample].round(2))

## 3. Fat Tails

Every currency pair shows heavier tails than Gaussian. Here we inspect EUR/USD
with a histogram and QQ-plot, then compare kurtosis across all currencies.

In [ ]:
returns = pd.read_csv(ANALYSIS / "daily_log_returns.csv")
eur = returns[returns["currency"] == "EUR"]["log_return"].values

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(eur, bins=200, density=True, alpha=0.7, color="#4C72B0", label="Observed")
from scipy.stats import norm
mu, sigma = eur.mean(), eur.std()
x = np.linspace(-0.05, 0.05, 500)
ax1.plot(x, norm.pdf(x, mu, sigma), "r-", lw=2, label="Normal fit")
ax1.set_xlim(-0.05, 0.05)
ax1.set_xlabel("Daily log return")
ax1.set_ylabel("Density")
ax1.set_title("EUR/USD: observed vs Gaussian")
ax1.legend()

# QQ-plot
(osm, osr), (slope, intercept, _) = probplot(eur, dist="norm")
ax2.scatter(osm, osr, s=3, alpha=0.5, color="#4C72B0")
xlim = ax2.get_xlim()
x_ref = np.linspace(xlim[0], xlim[1], 100)
ax2.plot(x_ref, slope * x_ref + intercept, "r-", lw=1.5)
ax2.set_xlabel("Theoretical quantiles")
ax2.set_ylabel("Observed quantiles")
ax2.set_title("EUR/USD: QQ-plot")

plt.tight_layout()
plt.show()

In [ ]:
# Kurtosis comparison across all currencies
kurt_data = []
for currency, grp in returns.groupby("currency"):
    r = grp["log_return"].values
    kurt_data.append({"currency": currency, "excess_kurtosis": kurtosis(r, fisher=True)})
kurt_df = pd.DataFrame(kurt_data).sort_values("excess_kurtosis", ascending=False)

fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(kurt_df["currency"], kurt_df["excess_kurtosis"], color="#4C72B0")
ax.set_xlabel("Excess kurtosis")
ax.set_title("Fat tails: excess kurtosis by currency (daily log returns)")
ax.set_xscale("log")
plt.tight_layout()
plt.show()

## 4. Regime Analysis

The Ilzetzki-Reinhart-Rogoff classification assigns each country-year a regime type
(peg, crawling peg, managed float, free float, freely falling, dual market).

In [ ]:
regimes = pd.read_csv(ANALYSIS / "yearly_regime_classification.csv")
print(f"Regime data: {len(regimes):,} country-year obs, {regimes['country'].nunique()} countries")
print(f"\nRegime distribution:")
print(regimes["regime_label"].value_counts())

In [ ]:
# Bar chart of regime distribution
counts = regimes["regime_label"].value_counts()
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(counts.index, counts.values, color="#4C72B0")
ax.set_ylabel("Country-year observations")
ax.set_title("Exchange rate regime distribution (1940-2019)")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# Volatility by regime
cond_stats = pd.read_csv(ANALYSIS / "regime_conditional_stats.csv")
print("Regime-conditional statistics:")
print(cond_stats[["regime", "n_observations", "volatility", "excess_kurtosis"]].to_string(index=False))

## 5. Gold Inflation

Gold purchasing power shows the long-run debasement of fiat currencies.
Cumulative retained percentage tells how much purchasing power a currency
has kept relative to its starting year.

In [ ]:
gold = pd.read_csv(ANALYSIS / "yearly_gold_inflation.csv")
print(f"Gold inflation data: {len(gold):,} rows, {gold['country'].nunique()} countries")
print(f"Year range: {gold['year'].min()}-{gold['year'].max()}")

In [ ]:
# Cumulative debasement for selected currencies
fig, ax = plt.subplots(figsize=(14, 6))
for country, color, label in [
    ("United States", "#4C72B0", "USD"),
    ("United Kingdom", "#DD8452", "GBP"),
    ("Japan", "#55A868", "JPY"),
    ("Switzerland", "#C44E52", "CHF"),
]:
    sub = gold[gold["country"] == country].sort_values("year")
    if len(sub) > 0:
        ax.plot(sub["year"], sub["cumulative_retained_pct"],
                label=f"{label} (since {int(sub['base_year'].iloc[0])})",
                color=color, linewidth=1.2)

ax.set_yscale("log")
ax.set_xlabel("Year")
ax.set_ylabel("% gold purchasing power retained (log)")
ax.set_title("Currency debasement against gold")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Medieval Data

The MEMDB Spufford dataset contains 13,197 exchange quotations from 1106-1500,
covering 521 places across Europe, Byzantium, the Levant, and North Africa.

In [ ]:
spufford = pd.read_csv(SOURCES / "memdb" / "memdb_spufford_medieval_exchange_rates.csv")
print(f"Spufford: {len(spufford):,} records")
print(f"Columns: {list(spufford.columns)}")
print(f"\nOldest records:")
print(spufford.sort_values("year").head(10).to_string(index=False))

In [ ]:
# Trading cities by record count
if "place" in spufford.columns:
    col = "place"
elif "from_place" in spufford.columns:
    col = "from_place"
else:
    col = spufford.columns[1]  # fallback

top_places = spufford[col].value_counts().head(20)
fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(top_places.index[::-1], top_places.values[::-1], color="#4C72B0")
ax.set_xlabel("Number of exchange quotations")
ax.set_title("Top 20 medieval trading cities by exchange records")
plt.tight_layout()
plt.show()